In [122]:
# ChatGPT modified version 2.0 - all methods have to be carefully reviewed.

import pandas as pd
import numpy as np
import os
import gzip
from io import StringIO
import re
import sys
from Bio import SeqIO
import pandas as pd


# Define global variables
cnv_count = 1
sv_count = 1
snv_count = 1

def increment_cnv():
    global cnv_count
    cnv_count += 1

def increment_sv():
    global sv_count
    sv_count += 1

def increment_snv():
    global snv_count
    snv_count += 1

def convert_chromosome(chrom):
    if chrom == 'chrX' or chrom == 'X':
        return "23"
    elif chrom == "chrY" or chrom == 'Y':
        return "24"
    elif chrom.startswith('chr'):
        return str(chrom[3:])
    else:
        return str(chrom)

def init_vcf(output_folder, sample):
    output_file_path = os.path.join(output_folder, sample + '.vcf')
    with open(output_file_path, 'w') as file_vcf:
        file_vcf.write('##fileformat=VCFv4.2\n')
        file_vcf.write('##filedate=20211011\n')
        file_vcf.write('##INFO=<ID=END,Number=1,Type=Integer,Description="End position of the variant described in this record">\n')
        file_vcf.write('##INFO=<ID=IMPRECISE,Number=0,Type=Flag,Description="Imprecise structural variation">\n')
        file_vcf.write('##INFO=<ID=MATEID,Number=.,Type=String,Description="ID of mate breakends">\n')
        file_vcf.write('##INFO=<ID=SVTYPE,Number=1,Type=String,Description="Type of structural variant">\n')
        file_vcf.write('##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">\n')
        file_vcf.write('##FORMAT=<ID=CN,Number=2,Type=Integer,Description="Copy number genotype for imprecise events">\n')
        file_vcf.write('##FORMAT=<ID=CNADJ,Number=.,Type=Float,Description="Copy number of adjacency">\n')
        file_vcf.write('##FORMAT=<ID=BDP,Number=1,Type=Integer,Description="Depth of split reads">\n')
        file_vcf.write('##FORMAT=<ID=DP,Number=1,Type=Integer,Description="Read depth">\n')
        file_vcf.write('##ALT=<ID=DEL,Description="Deletion">\n')
        file_vcf.write('##ALT=<ID=DUP,Description="Duplication">\n')
        file_vcf.write('##ALT=<ID=INS,Description="Insertion of novel sequence">\n')
        file_vcf.write('##ALT=<ID=CNV,Description="Copy number variable region">\n')
        file_vcf.write('#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\tTUMOR\tNORMAL\n')
    return output_file_path

def write_df_to_vcf(df, vcf_file):
    # Filter out rows where #CHROM column has the value 24
    # Ensure the #CHROM column is treated as a string
    df['#CHROM'] = df['#CHROM'].astype(str)
    
    # Perform the filtering operation
    filtered_df = df[df['#CHROM'] != '24']
    filtered_df.to_csv(vcf_file, sep='\t', mode='a', index=False, header=False)


def load_ascat_cnv(file_path):
    """
    Reads an ASCAT output file into a pandas DataFrame.

    Parameters:
    file_path (str): The path to the ASCAT output file.

    Returns:
    pd.DataFrame: A DataFrame containing the ASCAT data.
    """
    # Define column names as specified in the example
    column_names = ['chr', 'startpos', 'endpos', 'nMajor', 'nMinor']
    
    # Read the file into a DataFrame
    df = pd.read_csv(file_path, sep='\t', names=column_names, header=0)
    df['chr'] = df['chr'].apply(convert_chromosome)
    return df

def convert_ascat_to_vcf_df(df, reference_genome_path):
    """
    Converts a DataFrame with ASCAT data to a VCF compatible DataFrame,
    adding rows for all segments of each chromosome that are not already in the data.
    """
    vcf_records = []
    chromosome_lengths = {}
    cnv_count = 0



    # Parse the reference genome file
    for record in SeqIO.parse(reference_genome_path, "fasta"):
        chromosome = record.id
        length = len(record.seq)
        if chromosome.startswith("NC_0000"):
            chromosome_number = chromosome.split('.')[0].replace("NC_0000", "")
            chromosome_number = str(int(chromosome_number))  # Remove leading zeros
            chromosome_lengths[chromosome_number] = length
        elif chromosome.startswith("NC_012920.1"):
            # Skip the MT chromosome
            continue

    # Sort chromosome lengths based on chromosome number
    sorted_chromosome_lengths = dict(sorted(chromosome_lengths.items(), key=lambda item: int(item[0])))
    print(sorted_chromosome_lengths)
    # Process each chromosome
    for chrom, length in sorted_chromosome_lengths.items():
        if chrom == "24" and "24" not in df['chr'].values:
            continue

        # Map chromosome "23" to "X" and "24" to "Y" for data lookup
        lookup_chrom = "23" if chrom == "23" else ("24" if chrom == "24" else chrom)
        
        chrom_df = df[df['chr'] == str(lookup_chrom)]
        sorted_chrom_df = chrom_df.sort_values(by='startpos')
        
        last_end = 0
        print(sorted_chrom_df)
        # If no data for this chromosome, add a single entry covering the entire chromosome
        if sorted_chrom_df.empty:
            record_id = increment_cnv()
            info = f"END={length};IMPRECISE"
            format_field = "GT:CN"
            tumor_sample = "1|1:1,1"
            normal_sample = "0|0:1,1"
            vcf_records.append([chrom, 1, record_id, ".", "<CNV>", ".", "PASS", info, format_field, tumor_sample, normal_sample])
        else:
            for idx, row in sorted_chrom_df.iterrows():
                start = row['startpos']
                end = row['endpos']
                nMajor = row['nMajor']
                nMinor = row['nMinor']

                # Fill gap before this segment if needed
                if start > last_end + 1:
                    record_id = increment_cnv()
                    info = f"END={start-1};IMPRECISE"
                    format_field = "GT:CN"
                    tumor_sample = "1|1:1,1"
                    normal_sample = "0|0:1,1"
                    vcf_records.append([chrom, last_end + 1, record_id, ".", "<CNV>", ".", "PASS", info, format_field, tumor_sample, normal_sample])

                # Add the segment from ASCAT data
                record_id = increment_cnv()
                info = f"END={end};IMPRECISE"
                format_field = "GT:CN"
                tumor_sample = f"1|1:{nMajor},{nMinor}"
                normal_sample = "0|0:1,1"
                vcf_records.append([chrom, start, record_id, ".", "<CNV>", ".", "PASS", info, format_field, tumor_sample, normal_sample])

                last_end = end

            # Fill gap after last segment if needed
            if last_end < length:
                record_id = increment_cnv()
                info = f"END={length};IMPRECISE"
                format_field = "GT:CN"
                tumor_sample = "1|1:1,1"
                normal_sample = "0|0:1,1"
                vcf_records.append([chrom, last_end + 1, record_id, ".", "<CNV>", ".", "PASS", info, format_field, tumor_sample, normal_sample])

    # Create the VCF DataFrame
    vcf_df = pd.DataFrame(vcf_records, columns=["#CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO", "FORMAT", "TUMOR", "NORMAL"])
    
    # Ensure #CHROM and POS are int columns before sorting
    vcf_df["#CHROM"] = vcf_df["#CHROM"].astype(int)
    vcf_df["POS"] = vcf_df["POS"].astype(int)
    
    # Sort the VCF DataFrame by chromosome and position
    vcf_df = vcf_df.sort_values(by=["#CHROM", "POS"]).reset_index(drop=True)
    
    # Reassign CNV IDs in numerical ascending order
    vcf_df["ID"] = ["cnv" + str(i + 1) for i in range(len(vcf_df))]
    
    return vcf_df

def read_strelka(input_file_path):
    filtered_lines = []
    with gzip.open(input_file_path, 'rt') as file:
        for line in file:
            if not line.startswith('##'):
                filtered_lines.append(line)

    filtered_content = ''.join(filtered_lines)
    df_snv = pd.read_csv(StringIO(filtered_content), sep='\t')
    df_snv['#CHROM'] = df_snv['#CHROM'].apply(convert_chromosome)
    return df_snv

def get_position(input_string, search):
    # Split the input string by ':'
    split_string = input_string.split(':')
    
    # Find the position of 'AU'
    try:
        position = split_string.index(search)
    except ValueError:
        position = -1  # 'AU' not found in the string
    
    return position

def get_variant_reads(row, type, alt):

    AU_index = get_position(row['FORMAT'],'AU')
    CU_index = get_position(row['FORMAT'],'CU')
    GU_index = get_position(row['FORMAT'],'GU')
    TU_index = get_position(row['FORMAT'],'TU')
                
    if alt == "A":
        variant_reads = row[type].split(":")[AU_index].split(',')
        variant_reads = [int(x) for x in variant_reads] 
        total_variant_reads = sum(variant_reads)  # Sum the integer values
    elif alt == "C":
        variant_reads = row[type].split(":")[CU_index].split(',')
        variant_reads = [int(x) for x in variant_reads] 
        total_variant_reads = sum(variant_reads)  # Sum the integer values
    elif alt == "G":
        variant_reads = row[type].split(":")[GU_index].split(',')
        variant_reads = [int(x) for x in variant_reads] 
        total_variant_reads = sum(variant_reads)  # Sum the integer values
    elif alt == "T":
        variant_reads = row[type].split(":")[TU_index].split(',')
        variant_reads = [int(x) for x in variant_reads] 
        total_variant_reads = sum(variant_reads)  # Sum the integer values
    else:
        raise Exception("Not recognized nucleotide")

    indices = [AU_index, CU_index, GU_index, TU_index]
    total_reads = []
    
    for index in indices:
        total_reads += row['TUMOR'].split(":")[index].split(',')
                
    total_reads = [int(x) for x in total_reads]
    total_reads = sum(total_reads)
    
    return total_variant_reads, total_reads


def convert_strelka_snv_to_tusv_ext_df(strelka_df, ascat_df, option):
    """
    option can be phased or unphased
    """
    tusv_ext_rows = []
    strelka_df = strelka_df[strelka_df["FILTER"] == "PASS"]
    strelka_df.reset_index(inplace=True)
    for index, row in strelka_df.iterrows():
        chrom = row['#CHROM']
        pos = row['POS']
        ref = row['REF']
        alt = row['ALT']
        if option == "phased":
            raise Exception("Not implemented yet")
        elif option == "unphased":
            data_types = ["TUMOR", "NORMAL"]
            CNADJ_dict = {}
            for type in data_types:  
                total_variant_reads, total_reads = get_variant_reads(row, type, alt)
                CN_tot = get_total_copy_number(ascat_df, chrom, pos)
    
                
                CNADJ = (total_variant_reads)/(total_reads)*CN_tot
                CNADJ = float(CNADJ)
                CNADJ = format(CNADJ, '.2f')
                CNADJ_dict[type] = CNADJ
        else:
            raise Exception("Unknown Option")
        qual = "."
        filter_val = row["FILTER"]
        info = "."
        variant_id = f"snv{snv_count}"
        increment_snv()


        format_field = 'GT:CNADJ'
        tumor_data = "1|1" + ":" + CNADJ_dict["TUMOR"]
        normal_data = "0|0" + ":" + CNADJ_dict["NORMAL"]
        tusv_ext_rows.append([
            chrom, pos, variant_id, ref, alt, qual, filter_val, info, format_field, tumor_data, normal_data
        ])
        
    tusv_ext_df = pd.DataFrame(tusv_ext_rows, columns=[
        '#CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'TUMOR', 'NORMAL'
    ])
    return tusv_ext_df

def read_manta_sv(input_file_path):
    filtered_lines = []
    with gzip.open(input_file_path, 'rt') as file:
        for line in file:
            if not line.startswith('##'):
                filtered_lines.append(line)
    filtered_content = ''.join(filtered_lines)
    df_manta_sv = pd.read_csv(StringIO(filtered_content), sep='\t')
    df_manta_sv['#CHROM'] = df_manta_sv['#CHROM'].apply(convert_chromosome)
    
    return df_manta_sv



def get_total_copy_number(df, chrom, pos):
    """
    Gets the total copy number (major + minor) at a given chromosome and position.

    Parameters:
    df (pd.DataFrame): The DataFrame containing ASCAT data.
    chrom (str or int): The chromosome number.
    pos (int): The position on the chromosome.

    Returns:
    int or None: The total copy number at the given position, or None if the position is not covered.
    """
    # Filter the DataFrame for the given chromosome
    chr_df = df[df['chr'] == chrom]
    
    # Iterate over the rows to find the interval that contains the position
    for _, row in chr_df.iterrows():
        if row['startpos'] <= pos <= row['endpos']:
            total_cn = row['nMajor'] + row['nMinor']
            return total_cn
    
    # If no interval contains the position, return 2 assuming 2 normal alleles
    return 2


# Function to rename columns
def rename_column(col_name):
    last_split = col_name.split('-')[-1]
    if last_split.startswith('T'):
        return 'TUMOR'
    elif last_split.startswith('N'):
        return 'NORMAL'
    return col_name
# You may need to estimate CNAJ using CNVKit information tied to Manta output, and right hard conversion functions for DP and BDP


def calculate_metrics(ascat_df, row):
        # Ref then alt in the order list
    data_columns = ['TUMOR', 'NORMAL']

    for data in data_columns:
        if row["FORMAT"] == "PR":
            PR = row[data]
            
            PR_ref = int(PR.split(",")[0])
            PR_alt = int(PR.split(",")[1])
            PR_tot = PR_ref + PR_alt
            
            SR_ref = 0
            SR_alt = 0
            SR_tot = 0
        else:
            try:
        
                PR = row[data].split(":")[0]
                SR = row[data].split(":")[1]
        
                PR_ref = int(PR.split(",")[0])
                PR_alt = int(PR.split(",")[1])
                PR_tot = PR_ref + PR_alt
        
                
                SR_ref = int(SR.split(",")[0])
                SR_alt = int(SR.split(",")[1])
                SR_tot = SR_ref + SR_alt
    
            except IndexError:
                raise Exception("IndexError found")

        if data == 'TUMOR':
            GT = "1|1"
            CN_tot = get_total_copy_number(ascat_df, row['#CHROM'], row['POS'])
            
            CNADJ = (PR_alt + SR_alt)/(PR_tot + SR_tot)*CN_tot
            CNADJ = float(CNADJ)
            CNADJ_formatted = format(CNADJ, '.2f')

            BDP = SR_alt
            DP = SR_alt + PR_alt
            variables = [GT, CNADJ_formatted, BDP, DP]
            tumor_data = ':'.join(map(str, variables))
        elif data == 'NORMAL':
            GT = "0|0"
            CN_tot = 2
            CNADJ = (PR_ref + SR_ref)/(PR_tot + SR_tot)*CN_tot
            CNADJ = float(CNADJ)
            CNADJ_formatted = format(CNADJ, '.2f')
            BDP = SR_ref
            DP = SR_ref + PR_alt
            variables = [GT, CNADJ_formatted, BDP, DP]
            normal_data = ':'.join(map(str, variables))
        else:
            raise Exception("Unknown Datatype")
    return tumor_data, normal_data

def rewrite_manta_results(df_manta_sv, ascat_df):

    # Filter for SVs that pass all filters
    df_manta_sv = df_manta_sv[df_manta_sv['FILTER'] == "PASS"]

    
    # Filter for BND types and paired breakpoints
    bnd_df = df_manta_sv[df_manta_sv['INFO'].str.contains("SVTYPE=BND")]

    # Split the INFO column into a dictionary for easy access
    bnd_df['INFO_DICT'] = bnd_df['INFO'].apply(lambda x: dict(item.split('=') for item in x.split(';') if '=' in item))

    # Filter out unpaired BNDs
    mate_ids = bnd_df['INFO_DICT'].apply(lambda x: x.get('MATEID', None))
    paired_bnd_df = bnd_df[bnd_df['ID'].isin(mate_ids.values)]


    # Prepare new VCF columns
    vcf_columns = ['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'TUMOR', 'NORMAL']
    vcf_df = pd.DataFrame(columns=vcf_columns)
    
    # Create a dictionary to store the paired rows
    paired_rows = {}

    # Renaming column heads for ease of access downstream to TUMOR and NORMAL
    paired_bnd_df.rename(columns=lambda x: rename_column(x), inplace=True)
    print(paired_bnd_df['FORMAT'].unique())
    for index, row in paired_bnd_df.iterrows():
        mate_id = row['INFO_DICT']['MATEID']
        paired_rows[row['ID']] = row
        if mate_id in paired_rows:


            
            mate_row = paired_rows[mate_id]


            # Create the ALT field
            alt = row["ALT"]
            alt = str(alt).replace('chr','')
            alt = alt.replace(row['REF'],'')
            alt = alt.replace('X','23')
            #Recalculate ID here for row and mate_row
            row_sv_ID = f"sv{sv_count}"
            increment_sv()

            mate_row_sv_ID = f"sv{sv_count}"
            increment_sv()
            
            # Create the INFO field
            info = f"MATEID={mate_row_sv_ID};SVTYPE=BND"

            # Create the FORMAT fields 
            format_field = "GT:CNADJ:BDP:DP"


            tumor_data, normal_data = calculate_metrics(ascat_df, row)


            # Append the rows to the new DataFrame
            vcf_df = vcf_df.append({
                '#CHROM': row['#CHROM'], 'POS': row['POS'], 'ID': row_sv_ID, 'REF': '.',
                'ALT': alt, 'QUAL': row['QUAL'], 'FILTER': row['FILTER'], 'INFO': info,
                'FORMAT': format_field, 'TUMOR': tumor_data, 'NORMAL': normal_data
            }, ignore_index=True)


            # Create the ALT field
            alt_mate = mate_row["ALT"]
            alt_mate = str(alt_mate).replace('chr','')
            alt_mate = alt_mate.replace(mate_row['REF'],'')
            alt_mate = alt_mate.replace('X','23')


            info_mate = f"MATEID={row_sv_ID};SVTYPE=BND"


            tumor_data, normal_data = calculate_metrics(ascat_df, mate_row)

            
            vcf_df = vcf_df.append({
                '#CHROM': mate_row['#CHROM'], 'POS': mate_row['POS'], 'ID': mate_row_sv_ID, 'REF': '.',
                'ALT': alt_mate, 'QUAL': mate_row['QUAL'], 'FILTER': mate_row['FILTER'], 'INFO': info_mate,
                'FORMAT': format_field, 'TUMOR': tumor_data, 'NORMAL': normal_data
            }, ignore_index=True)

    return vcf_df
    
def main():
    global cnv_count, sv_count, snv_count
    cnv_count = 1
    sv_count = 1
    snv_count = 1
    variant_calling_folder = '/bgfs/alee/LO_LAB/Personal/Alexander_Chang/alc376/TUSV-ext/AER5_wes_variant_calling/'
    output_folder = '/bgfs/alee/LO_LAB/Personal/Alexander_Chang/alc376/TUSV-ext/compiled_input/'
    sample_name = 'AUR-AER5-TTM4_vs_AUR-AER5-NT2'
    frozen_samples = ['AUR-AER5-TTM4_vs_AUR-AER5-NT2', 'AUR-AER5-TTM5_vs_AUR-AER5-NT2', 'AUR-AER5-TTP2_vs_AUR-AER5-NT2']
    # ASCAT_JOINT_MUTECT

    # Path to the GRCh38 reference genome file
    reference_genome_path = "/bgfs/alee/LO_LAB/General/References/GCF_GRCh38/GCF_000001405.40_GRCh38.p14_genomic.fna"

    
    for sample_name in frozen_samples:
        print(sample_name)
        strelka_snv_path = os.path.join(variant_calling_folder, 'strelka', sample_name, sample_name + '.strelka.somatic_snvs.vcf.gz')
        strelka_indel_path = os.path.join(variant_calling_folder, 'strelka', sample_name, sample_name + '.strelka.somatic_indels.vcf.gz')
        manta_sv_path = os.path.join(variant_calling_folder, 'manta', sample_name, sample_name + '.manta.somatic_sv.vcf.gz')
        ascat_cnv_path = os.path.join(variant_calling_folder, 'ascat', sample_name, sample_name + '.cnvs.txt')
    
    
        output_file_path = init_vcf(output_folder, sample_name)
    
        
        ascat_df = load_ascat_cnv(ascat_cnv_path)
        vcf_df = convert_ascat_to_vcf_df(ascat_df, reference_genome_path)
        write_df_to_vcf(vcf_df, output_file_path)
    
    
        df_manta_sv = read_manta_sv(manta_sv_path)
        df_processed_manta = rewrite_manta_results(df_manta_sv, ascat_df)
        write_df_to_vcf(df_processed_manta, output_file_path)
    
        df_snv = read_strelka(strelka_snv_path)
        tusv_ext_snv = convert_strelka_snv_to_tusv_ext_df(df_snv, ascat_df, "unphased")
        write_df_to_vcf(tusv_ext_snv, output_file_path)

    # TODO: write assertion checks for the final document. 

if __name__ == "__main__":
    main()


AUR-AER5-TTM4_vs_AUR-AER5-NT2
{'1': 248956422, '2': 242193529, '3': 198295559, '4': 190214555, '5': 181538259, '6': 170805979, '7': 159345973, '8': 145138636, '9': 138394717, '10': 133797422, '11': 135086622, '12': 133275309, '13': 114364328, '14': 107043718, '15': 101991189, '16': 90338345, '17': 83257441, '18': 80373285, '19': 58617616, '20': 64444167, '21': 46709983, '22': 50818468, '23': 156040895, '24': 57227415}
  chr   startpos     endpos  nMajor  nMinor
0   1     873251   35918614       2       1
1   1   36100398  125122318       2       2
2   1  145837693  160796870       6       1
3   1  160797135  161379107      11       1
4   1  161509955  167683765       6       1
5   1  167765556  248916508       2       1
   chr   startpos     endpos  nMajor  nMinor
6    2      41404  119246204       3       2
7    2  119301482  124764368       3       1
8    2  124789687  128797876       3       2
9    2  129496404  192699777       3       1
10   2  193277860  241744700       3       0


/bgfs/alee/LO_LAB/Personal/Alexander_Chang/alc376/envs/tusv_ext/lib/python3.6/site-packages/ipykernel_launcher.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


AUR-AER5-TTM5_vs_AUR-AER5-NT2
{'1': 248956422, '2': 242193529, '3': 198295559, '4': 190214555, '5': 181538259, '6': 170805979, '7': 159345973, '8': 145138636, '9': 138394717, '10': 133797422, '11': 135086622, '12': 133275309, '13': 114364328, '14': 107043718, '15': 101991189, '16': 90338345, '17': 83257441, '18': 80373285, '19': 58617616, '20': 64444167, '21': 46709983, '22': 50818468, '23': 156040895, '24': 57227415}
  chr   startpos     endpos  nMajor  nMinor
0   1     873251   35918614       2       1
1   1   36100398  125122318       2       2
2   1  145837693  160670822       4       1
3   1  160796868  161379107       8       1
4   1  161509955  164935503       4       1
5   1  165204312  167854735       3       1
6   1  167856247  248916508       2       1
  chr   startpos     endpos  nMajor  nMinor
7   2      41404   70165247       3       2
8   2   70165688  103825205       4       2
9   2  103825227  241744700       4       1
   chr   startpos     endpos  nMajor  nMinor
10   

/bgfs/alee/LO_LAB/Personal/Alexander_Chang/alc376/envs/tusv_ext/lib/python3.6/site-packages/ipykernel_launcher.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


AUR-AER5-TTP2_vs_AUR-AER5-NT2
{'1': 248956422, '2': 242193529, '3': 198295559, '4': 190214555, '5': 181538259, '6': 170805979, '7': 159345973, '8': 145138636, '9': 138394717, '10': 133797422, '11': 135086622, '12': 133275309, '13': 114364328, '14': 107043718, '15': 101991189, '16': 90338345, '17': 83257441, '18': 80373285, '19': 58617616, '20': 64444167, '21': 46709983, '22': 50818468, '23': 156040895, '24': 57227415}
  chr   startpos     endpos  nMajor  nMinor
0   1     873251   35918614       3       0
1   1   36100398   36286551       4       1
2   1   36290034   60180520       4       0
3   1   60476692  125122318       4       1
4   1  145837693  160670822       3       3
5   1  160796868  161379107       6       3
6   1  161509955  167854274       3       3
7   1  167854734  248916508       3       2
   chr   startpos     endpos  nMajor  nMinor
8    2      41404   19102061       1       0
9    2   19347034  164493392       2       1
10   2  164494698  166046718       2       0
11

/bgfs/alee/LO_LAB/Personal/Alexander_Chang/alc376/envs/tusv_ext/lib/python3.6/site-packages/ipykernel_launcher.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [83]:
import vcfpy


def is_cnv_record(rec):
    return rec.ID[0][0:3] == 'cnv'


def is_sv_record(rec):
    return rec.ID[0][0:2] == 'sv'


def is_snv_record(rec):
    return rec.ID[0][0:3] == 'snv'


vcf_reader =  vcfpy.Reader.from_path('/bgfs/alee/LO_LAB/Personal/Alexander_Chang/alc376/TUSV-ext/compiled_input/AUR-AER5-TTM4_vs_AUR-AER5-NT2.vcf')

for rec in vcf_reader:
    if is_sv_record(rec):
        print(rec.ALT[0])
        print(rec.ALT[0])

BreakEnd('1', 185147263, '+', '-', '', True)
BreakEnd('1', 185147263, '+', '-', '', True)
BreakEnd('3', 116437976, '+', '+', '', True)
BreakEnd('3', 116437976, '+', '+', '', True)
BreakEnd('2', 43710295, '+', '+', 'T', True)
BreakEnd('2', 43710295, '+', '+', 'T', True)
BreakEnd('6', 41633110, '+', '+', 'A', True)
BreakEnd('6', 41633110, '+', '+', 'A', True)
BreakEnd('4', 156027395, '+', '+', '', True)
BreakEnd('4', 156027395, '+', '+', '', True)
BreakEnd('7', 4920148, '+', '+', '', True)
BreakEnd('7', 4920148, '+', '+', '', True)
BreakEnd('6', 72182816, '+', '-', '', True)
BreakEnd('6', 72182816, '+', '-', '', True)
BreakEnd('7', 65845265, '+', '-', '', True)
BreakEnd('7', 65845265, '+', '-', '', True)
BreakEnd('4', 1701063, '-', '+', 'AACA', True)
BreakEnd('4', 1701063, '-', '+', 'AACA', True)
BreakEnd('7', 100205251, '+', '-', 'GGAGACA', True)
BreakEnd('7', 100205251, '+', '-', 'GGAGACA', True)
BreakEnd('1', 145831220, '+', '-', '', True)
BreakEnd('1', 145831220, '+', '-', '', True)


In [100]:
import os

output_folder = '/bgfs/alee/LO_LAB/Personal/Alexander_Chang/alc376/TUSV-ext/compiled_input/'


sampleList = os.listdir(output_folder)
sv_ub = 80
max_sv = -1
for i, sample in enumerate(sampleList):
    sv_count = 0
    input_vcf_file = output_folder  + sample
    reader =  vcfpy.Reader.from_path(input_vcf_file)
    for rec in reader:
        if is_sv_record(rec):
            sv_count += 1
    max_sv = max(max_sv, sv_count)
if max_sv >= sv_ub:
    print(f"There are more SVs in one or more samples than your SV Upperbound, increasing SV_UB to {max_sv}, and C to {max_sv+40}")
    sv_ub = max_sv
    const = max_sv+40



There are more SVs in one or more samples than your SV Upperbound, increasing SV_UB to 184, and C to 224


In [106]:
from Bio import SeqIO

# Path to the GRCh38 reference genome file
reference_genome_path = "/bgfs/alee/LO_LAB/General/References/GCF_GRCh38/GCF_000001405.40_GRCh38.p14_genomic.fna"

# Dictionary to store chromosome lengths
chromosome_lengths = {}

# Parse the reference genome file
for record in SeqIO.parse(reference_genome_path, "fasta"):
    chromosome = record.id
    length = len(record.seq)
    if chromosome.startswith("NC_0000") or chromosome.startswith("NC_012920.1"):
        chromosome_number = chromosome.split('.')[0].replace("NC_0000", "").replace("NC_012920", "MT")
        chromosome_lengths[chromosome_number] = length

# Sort chromosome lengths based on chromosome number
sorted_chromosome_lengths = dict(sorted(chromosome_lengths.items(), key=lambda item: int(item[0].replace("MT", "25").replace("X", "23").replace("Y", "24"))))

# Print the lengths of the main chromosomes
for chromosome, length in sorted_chromosome_lengths.items():
    print(f"Chromosome {chromosome}: {length} bp")


Chromosome 01: 248956422 bp
Chromosome 02: 242193529 bp
Chromosome 03: 198295559 bp
Chromosome 04: 190214555 bp
Chromosome 05: 181538259 bp
Chromosome 06: 170805979 bp
Chromosome 07: 159345973 bp
Chromosome 08: 145138636 bp
Chromosome 09: 138394717 bp
Chromosome 10: 133797422 bp
Chromosome 11: 135086622 bp
Chromosome 12: 133275309 bp
Chromosome 13: 114364328 bp
Chromosome 14: 107043718 bp
Chromosome 15: 101991189 bp
Chromosome 16: 90338345 bp
Chromosome 17: 83257441 bp
Chromosome 18: 80373285 bp
Chromosome 19: 58617616 bp
Chromosome 20: 64444167 bp
Chromosome 21: 46709983 bp
Chromosome 22: 50818468 bp
Chromosome 23: 156040895 bp
Chromosome MT: 16569 bp
Chromosome 24: 57227415 bp


import vcf

vcf_reader = vcf.Reader(open('/bgfs/alee/LO_LAB/Personal/Alexander_Chang/alc376/TUSV-ext/compiled_input/AUR-AER5-TTM4_vs_AUR-AER5-NT2.vcf', 'r'))

for record in vcf_reader:
    print(record)

In [ ]:
# need to add an option so SNVs are given SNV IDs but strelka indels are given SV ids

In [17]:
# Test box for reading in Strelka indel files to test

test_strelka_indel_path = '/bgfs/alee/LO_LAB/Personal/Alexander_Chang/alc376/wholebody_phylo/AER5_results/variant_calling/strelka/AUR-AER5-TTM4_vs_AUR-AER5-NT2/AUR-AER5-TTM4_vs_AUR-AER5-NT2.strelka.somatic_snvs.vcf.gz'

df_indel = read_strelka(test_strelka_indel_path)


print(df_indel.head())
tusv_ext_indel = convert_strelka_to_tusv_ext_df(df_indel, "INDEL")
print(tusv_ext_indel.head())


  #CHROM    POS ID REF ALT QUAL  FILTER  \
0   chr1  10120  .   T   C    .  LowEVS   
1   chr1  10257  .   A   C    .  LowEVS   
2   chr1  10330  .   C   A    .  LowEVS   
3   chr1  14063  .   G   A    .  LowEVS   
4   chr1  14653  .   C   T    .  LowEVS   

                                                INFO  \
0  DP=1080;MQ=12.08;MQ0=874;NT=ref;QSS=8;QSS_NT=8...   
1  DP=190;MQ=16.38;MQ0=92;NT=ref;QSS=3;QSS_NT=3;R...   
2  DP=212;MQ=21.15;MQ0=89;NT=ref;QSS=1;QSS_NT=1;R...   
3  DP=37;MQ=18.01;MQ0=25;NT=ref;QSS=1;QSS_NT=1;Re...   
4  DP=45;MQ=29.11;MQ0=14;NT=ref;QSS=4;QSS_NT=4;Re...   

                         FORMAT                         NORMAL  \
0  DP:FDP:SDP:SUBDP:AU:CU:GU:TU  48:18:1:0:2,19:0,2:0,2:28,112   
1  DP:FDP:SDP:SUBDP:AU:CU:GU:TU      12:7:1:0:5,41:0,5:0,0:0,0   
2  DP:FDP:SDP:SUBDP:AU:CU:GU:TU     26:19:6:0:0,5:7,23:0,1:0,1   
3  DP:FDP:SDP:SUBDP:AU:CU:GU:TU       3:0:0:0:0,0:0,0:3,14:0,0   
4  DP:FDP:SDP:SUBDP:AU:CU:GU:TU     14:0:0:0:0,0:12,18:0,0:2,3   

       

In [18]:
# Test box for reading in Strelka snv files to test.
import sys

test_strelka_snv_path = '/bgfs/alee/LO_LAB/Personal/Alexander_Chang/alc376/wholebody_phylo/AER5_results/variant_calling/strelka/AUR-AER5-TTM4_vs_AUR-AER5-NT2/AUR-AER5-TTM4_vs_AUR-AER5-NT2.strelka.somatic_indels.vcf.gz'

df_snv = read_strelka(test_strelka_snv_path)

tusv_ext_snv = convert_strelka_to_tusv_ext_df(df_snv, "SNV")
print(tusv_ext_snv.head())


(14497, 13)
(5411, 13)
  #CHROM      POS       ID REF               ALT QUAL FILTER INFO    FORMAT  \
0   chr1  1056668  snv0000   C               CGT    .   PASS    .  GT:CNADJ   
1   chr1  1423775  snv0001   C                CT    .   PASS    .  GT:CNADJ   
2   chr1  1509914  snv0002  TA                 T    .   PASS    .  GT:CNADJ   
3   chr1  2900016  snv0003   C                CG    .   PASS    .  GT:CNADJ   
4   chr1  3329041  snv0004   G  GCCTTCTCCCTGGGCC    .   PASS    .  GT:CNADJ   

   TUMOR NORMAL  
0  0|1:0  0|0:0  
1  0|1:0  0|0:0  
2  0|1:0  0|0:0  
3  0|1:0  0|0:0  
4  0|1:0  0|0:0  


In [27]:
# Test box for reading in CNVKit .cns files to test.

test_cnvkit_cns_file = '/bgfs/alee/LO_LAB/Personal/Alexander_Chang/alc376/wholebody_phylo/AER5_results/variant_calling/cnvkit/AUR-AER5-TTM4_vs_AUR-AER5-NT2/AUR-AER5-TTM4.cns'

cnvkit_df = load_cnvkit_cns(test_cnvkit_cns_file)
print(cnvkit_df.head())

vcf_df = cnvkit_to_df(cnvkit_df)
print(vcf_df.head())



  chromosome    start       end gene      log2     depth  probes     weight  \
0       chr1    14034    351968    -  0.098452  14.55930      58    52.1960   
1       chr1   351968    459979    - -0.413539   7.87716      27    19.1118   
2       chr1   459979   1264807    - -0.022918  13.51170     180   170.4730   
3       chr1  1264807   1333091    -  0.322362  17.72340      17    16.8359   
4       chr1  1333091  16558479    - -0.011359  13.51330    3752  3682.8400   

      ci_lo     ci_hi  
0  0.080594  0.119033  
1 -0.444376 -0.386697  
2 -0.031888 -0.010873  
3  0.294485  0.344509  
4 -0.013140 -0.009817  
  #CHROM      POS     ID REF    ALT QUAL FILTER                    INFO  \
0   chr1    14034  cnv00   .  <CNV>    .   PASS    END=351968;IMPRECISE   
1   chr1   351968  cnv01   .  <CNV>    .   PASS    END=459979;IMPRECISE   
2   chr1   459979  cnv02   .  <CNV>    .   PASS   END=1264807;IMPRECISE   
3   chr1  1264807  cnv03   .  <CNV>    .   PASS   END=1333091;IMPRECISE   
4   ch

In [11]:
# Test box for reading in Manta .vcf files to determine format.

test_manta_file = '/bgfs/alee/LO_LAB/Personal/Alexander_Chang/alc376/wholebody_phylo/AER5_results/variant_calling/manta/AUR-AER5-TTM4_vs_AUR-AER5-NT2/AUR-AER5-TTM4_vs_AUR-AER5-NT2.manta.somatic_sv.vcf.gz'
test_organoid_wes_file = '/bgfs/alee/LO_LAB/Personal/Alexander_Chang/alc376/nfcore/achang4_bridgesPSC/sarek/Part2_FASTQ2SAREK/results/variant_calling/manta/123_organoid_vs_123_blood/123_organoid_vs_123_blood.manta.somatic_sv.vcf.gz'
test_manta_wes_file = '/bgfs/alee/LO_LAB/Personal/Alexander_Chang/alc376/TUSV-ext/variant_calling/manta/AUR-AER5-TTM4_vs_AUR-AER5-NT2/AUR-AER5-TTM4_vs_AUR-AER5-NT2.manta.somatic_sv.vcf.gz'

df_processed_manta = read_manta_sv(test_manta_wes_file)

print(df_processed_manta.head(10))

  #CHROM       POS                               ID REF                 ALT  \
0   chr1   3722167       MantaBND:11184:1:3:0:0:0:1   A     A]chr1:3722443]   
1   chr1   3722434       MantaBND:11184:1:3:0:0:0:0   A     A]chr1:3722176]   
2   chr1  10413332       MantaBND:73506:1:1:6:0:0:1   T    T]chr1:10413436]   
3   chr1  10413428       MantaBND:73506:1:1:6:0:0:0   T    T]chr1:10413340]   
4   chr1  15349115       MantaBND:87210:0:0:1:0:0:1   A    A]chr1:15349202]   
5   chr1  15349194       MantaBND:87210:0:0:1:0:0:0   G    G]chr1:15349123]   
6   chr1  16996026   MantaBND:1:97014:97014:1:0:0:0   G    [chr1:16996118[G   
7   chr1  16996113   MantaBND:1:97014:97014:1:0:0:1   C    [chr1:16996031[C   
8   chr1  20792077   MantaBND:1:31559:31577:0:0:0:1   G  GT]chr18:37273578]   
9   chr1  20867797  MantaDUP:TANDEM:12529:1:7:0:1:0   A        <DUP:TANDEM>   

  QUAL           FILTER                                               INFO  \
0    .  MinSomaticScore  SVTYPE=BND;MATEID=MantaBND:

In [ ]:
# Proceed with WES